In [ ]:
# -*- coding: utf-8 -*-
"""bi-LSTM.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Phkwb13SQ9Z4DyecE7qndGBulMKJwfXT
"""
sentence = [] #문장
category = [] #카테고리
vocab_size = 0
def read_file(file_name,index):
    import re
    from konlpy.tag import Okt
    global sentence,category
    f = open(file_name, "r", encoding='utf-8')
    list1 = []
    okt = Okt()
    for i in range(1, 10000):
        line = f.readline()  # 한 줄씩 읽음.
        if not line: break  # 모두 읽으면 while문 종료.
        line = re.sub('[^ㄱ-ㅣ가-힝0-9a-zA-Z\\s]', '', line)  # 특수문자제거를 위해 한글과 알파벳만 남기기.
        noun = okt.nouns(line)  # 명사추출
        stop_words = ['것', '수', '거', '곳', '저', '안', '제', '더', '때', '이', '진짜', '바로', '정말',
                      '여기', '개', '분', '정도', '그', '요', '중', '밤', '그', '요', '중', '위', '나', '내',
                      '가장', '게', '점', '좀', '또', '달', '말', '해', '은', '향', '번', '날', '아주', '완전', '꼭', '듯',
                      '그냥', '조금', '듯', '층', '사실', '도', '뭐', '살', '살짝', '걸', '쪽', '얼', '만', '꽤', '후']
        temp = [each_word for each_word in noun if each_word not in stop_words] #불용어 제거
        if temp:  # temp에 결과가 존재할 경우에만 (길이 0인거는 제외)
            list1.append(temp)
    f.close()
    sentence = sentence + list1
    cate = [index for k in range(len(list1))]
    category = category + cate

def proceed():
    global sentence, category,vocab_size
    from keras.preprocessing.text import Tokenizer
    t = Tokenizer()
    t.fit_on_texts(sentence) #사전 만들기
    vocab_size = len(t.word_index) + 1 #단어들의 개수
    print("vocab_size: ",vocab_size)
    X_encoded = t.texts_to_sequences(sentence) # 정수 인코딩

    max_len = max(len(l) for l in X_encoded)
    print("max_len :", max_len) #단어들의 최대개수

    # 모든 문장을 패딩하여 길이 최대 길이로 만들어줍니다.
    from keras.preprocessing.sequence import pad_sequences
    predData = pad_sequences(X_encoded, maxlen=max_len, padding='post')
    #print(predData)

    return predData


def modeling1():
    # MODELING 3
    # https://www.tensorflow.org/tutorials/text/text_classification_rnn
    global vocab_size
    from keras.models import Sequential
    from keras.layers import Dense, LSTM, Bidirectional, Embedding
    from keras.optimizers import Adam

    model = Sequential([
        Embedding(vocab_size, 64, mask_zero=True),
        Bidirectional(LSTM(64)),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax') #결과갯수
    ])
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(1e-4),
                  metrics=['accuracy'])
    model.summary()
    return model

def modeling2():
    # MODELING 4
    # https://www.tensorflow.org/tutorials/text/text_classification_rnn
    global vocab_size
    from keras.models import Sequential
    from keras.layers import Dense, LSTM, Dropout, Bidirectional, Embedding
    from keras.optimizers import Adam

    model = Sequential([
        Embedding(vocab_size, 64, mask_zero=True),
        Bidirectional(LSTM(64, return_sequences=True)),
        Bidirectional(LSTM(32)),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(3, activation='softmax') #결과갯수
    ])
    model.compile(loss='binary_crossentropy',
                  optimizer=Adam(1e-4),
                  metrics=['accuracy'])
    model.summary()
    return model

In [ ]:
#파일 읽기
read_file("/text/desrt(1006).txt", 0)
read_file("/text/food(974).txt", 1)
read_file("/text/recipi(997).txt", 2)
#전처리
predData = proceed()

# train 데이터와 test 데이터로 나누어줍니다.
from sklearn.model_selection import train_test_split
X_train, X_test, train_labels, test_labels = train_test_split(predData, category, test_size=0.2,random_state=321)  # 8:2 #3,4번째 인자 바꾸어주면됨
from keras.utils.np_utils import to_categorical
Y_train = to_categorical(train_labels)  # 라벨데이터 원 핫 인코딩(0-9사이의값(?))
Y_test = to_categorical(test_labels)

In [ ]:
#모델
#model = modeling1() #테스트 정확도: 0.6667
model = modeling2() # 테스트 정확도: 0.6941
history = model.fit(X_train, Y_train, batch_size=128, epochs=6)
#테스트 데이터
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, Y_test)[1]))

In [ ]:
model.save("/content/drive/My Drive/koLab/trained_model.h5", include_optimizer=False)